# SARSA & Q - Learning

# 연습데이터 : Frozen Lake
![좋은거](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSKrspq5EvLOC2Rv8dfsw1OVqD1WjN5YsmgAHVCVn3x8zLhM5Y8&usqp=CAU)

**학습목표**

1. SARSA의 업데이트 맛 본다.
2. Q-learning의 업데이트를 맛 본다.
3. 일단 코드 짤 수 있다.

# 미리 알아야 했지만..

[미리 알면 참 좋은 것. 1](https://image.slidesharecdn.com/reinforcementlearning-170904070210/95/introduction-of-deep-reinforcement-learning-22-638.jpg?cb=1504578048)

[역시 알아야 참 좋은 것. 2](http://www.modulabs.co.kr/files/attach/images/334/237/003/18eba72dcfeafa6e6280055a95078ffa.png)

# 필요 라이브러리 불러오기

1. 이런 연습에서는 딱히 비디오 영상이 필요하지 않다.

In [ ]:
!pip install gym

In [ ]:
import numpy as np
import gym

# 사용할 환경 불러오기

In [ ]:
gym.envs.registration.register(
    id="FrozenLake-v3", entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name': '4x4', 'is_slippery': False}
)

# gym.envs.registration.register(
#     id="FrozenLake-v8", entry_point='gym.envs.toy_text:FrozenLakeEnv',
#     kwargs={'map_name': '8x8', 'is_slippery': True}
# )

In [ ]:
env = gym.make('FrozenLake-v3')
# env = gym.make('FrozenLake-v8')
print('observation space:', env.observation_space)
print('action space:', env.action_space)

# SARSA와, Q-learning 준비 해보기

**핵심 코드**
1. SARSA : S,A,R,S,A 가 끝나면 Q업데이트!
```
Q[s,a] = Q[s,a] + alpha * ((r + gama* Q[s1,a1]) - Q[s,a])
```
2. Q-Learning : S,A,R,S 후 max A, 그다음 Q업데이트!
```
Q[s,a] = Q[s,a] + alpha*(r + gama*np.max(Q[s1,:]) - Q[s,a])
```


# I. SARSA

### 1. Q-Table을 준비하자.

* Q = np.zeros([state의 수, action의 수])
* 활용 예시
    * Q[2, 3]에는 10이 담겨있음.
    * Q[2, 1]에는 5가 담겨있음.
    * 2번 state에선 3번 액션이 1번 액션보다 2배 가치 있음!

* env.action_space.n, env.observation_space.n 이용

In [ ]:
## Your Code Here




In [ ]:
print(Q)
# for rows in Q.reshape([4,4,4]):
#     print(*rows)

### 2. parameter들

In [ ]:
alpha = 0.2
gamma = 0.999 # 할인율!
n_episod = 20000
epsilon = 0.15

### 3. SARSA!

In [ ]:
Q = np.zeros([env.observation_space.n,
              env.action_space.n])

rewards_sarsa = []

for i in range(n_episod) : 
    s0 = env.reset()
    done = False
    a0 = env.action_space.sample() # 처음엔 랜덤무브

    while True :
        s1, r1, done, _ = env.step(a0) # 일단 움직인다.

        if np.random.uniform() < epsilon :  # 무슨 뜻?
            a1 = env.action_space.sample()
        else : 
            greedy_actions = np.argwhere(Q[s1, :] == np.amax(Q[s1, :])).reshape(-1)
            a1 = np.random.choice(greedy_actions) # 무슨 뜻?

        # Update 한다!
        # ( s0, a0, r1, s1, a1 ) 로 부터!
        
        Q[s0, a0] = Q[s0, a0] + alpha * (r1 + gamma*Q[s1, a1] - Q[s0, a0])

        if done == True: # 종료되었으면
            rewards_sarsa.append(r1)
            env.close() # 환경 닫고.
            break # 멈춰야지

        s0 = s1 # 다음 루프에선 이것이 직전 state
        a0 = a1 # 다음 루프에선 이것이 직전 action

    if (i+1) % 4000 == 0 :
        print('===========  에피소드 : {}  ============'.format(i+1))
        for rows in Q.reshape([4,4,4]):
            print(*rows)

### 학습 완료된 것 구경
### 무한 루프 주의
위 에서, 학습을 1 ~ 2회만 시켜보고 아래를 해봐도 좋다.

In [ ]:
s0 = env.reset()
done = False
a0 = env.action_space.sample() # 처음엔 랜덤무브

while True :
    env.render()
    s1, r1, done, _ = env.step(a0) # 일단 움직인다.

    greedy_actions = np.argwhere(Q[s1, :] == np.amax(Q[s1, :])).reshape(-1)
    a1 = np.random.choice(greedy_actions) # 무슨 뜻?

    # 무한 루프 걸린다면 위의 두줄 주석, 아래를 주석 해제
    # if np.random.uniform() < epsilon :  # 무슨 뜻?
    #     a1 = env.action_space.sample()
    # else : 
    #     greedy_actions = np.argwhere(Q[s1, :] == np.amax(Q[s1, :])).reshape(-1)
    #     a1 = np.random.choice(greedy_actions) # 무슨 뜻?


    if done == True: # 종료되었으면
        env.render()
        env.close() # 환경 닫고.
        break # 멈춰야지

    s0 = s1 # 다음 루프에선 이것이 직전 state
    a0 = a1 # 다음 루프에선 이것이 직전 action

### SARSA 학습코드를 보고, 진행 순서를 정리해보자.

# II. Q-Learning


### 1. Q-Table을 준비하자.

* Q = np.zeros([state의 수, action의 수])
* 활용 예시
    * Q[2, 3]에는 10이 담겨있음.
    * Q[2, 1]에는 5가 담겨있음.
    * 2번 state에선 3번 액션이 1번 액션보다 2배 가치 있음!

* env.action_space.n, env.observation_space.n 이용

In [ ]:
env.observation_space, env.action_space

In [ ]:
## Your Code Here




In [ ]:
print(Q)
# for rows in Q.reshape([4,4,4]):
#     print(*rows)

### 2. parameter들

In [ ]:
alpha = 0.2
gamma = 0.999 # 할인율!
n_episod = 20000
epsilon = 0.15

### 3. q-learning

In [ ]:
Q = np.zeros([env.observation_space.n,
              env.action_space.n])

rewards_ql = []

for i in range(n_episod) : 
    s0 = env.reset()
    done = False

    while True :
        # 행동을 선택
        if np.random.uniform() < epsilon :  # 무슨 뜻?
            a0 = env.action_space.sample()
        else : 
            greedy_actions = np.argwhere(Q[s0, :] == np.amax(Q[s0, :])).reshape(-1)
            a0 = np.random.choice(greedy_actions) # 무슨 뜻?

        # a0 를 이용하여, 환경과 상호작용하고, s1, r1, done, _를 선언하시오.
        s1, r1, done, _ = env.step(a0)
        
        # ( s0, a0, r1, s1 ) 로 부터! 차이점 주의!
        Q[s0, a0] = Q[s0, a0] + alpha * (r1 + gamma*np.max(Q[s1,:]) - Q[s0, a0])

        if done == True: # 종료되었으면
            rewards_ql.append(r1)
            env.close() # 환경 닫고.
            break # 멈춰야지

        s0 = s1 # 다음 루프에선 이것이 직전 state

    if (i+1) % 4000 == 0 :
        print('===========  에피소드 : {}  ============'.format(i+1))
        for rows in Q.reshape([4,4,4]):
            print(*rows)
        

### 학습 완료된 것 구경

위 에서, 학습을 1 ~ 2회만 시켜보고 아래를 해봐도 좋다.

In [ ]:
s0 = env.reset()
done = False
a0 = env.action_space.sample() # 처음엔 랜덤무브

while True :
    env.render()
    s1, r1, done, _ = env.step(a0) # 일단 움직인다.

    greedy_actions = np.argwhere(Q[s1, :] == np.amax(Q[s1, :])).reshape(-1)
    a1 = np.random.choice(greedy_actions) # 무슨 뜻?

    if done == True: # 종료되었으면
        env.render()
        env.close() # 환경 닫고.
        break # 멈춰야지

    s0 = s1 # 다음 루프에선 이것이 직전 state
    a0 = a1 # 다음 루프에선 이것이 직전 action

### Q-learning 학습 코드를 보고 진행 순서를 정리하자.

# Bonus 시각화 :

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
win_ratio_s = []
win_ratio_ql = []

sliding_windows = 100
for i in range(n_episod - 100 + 1) :
    wr_s = sum(rewards_sarsa[i:i+sliding_windows])/sliding_windows
    wr_q = sum(rewards_ql[i:i+sliding_windows])/sliding_windows
    win_ratio_s.append(wr_s)
    win_ratio_ql.append(wr_q)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(win_ratio_s, label='SARSA')
plt.plot(win_ratio_ql, label='Q-Learning')
plt.legend()
plt.show()